This notebook reproduces the computational examples in Figures 3-5 from the manuscript.

In [ ]:
# add path to code
import sys
sys.path.insert(0, '../code')
import numpy as np

First, we specify that we are inverting for the basal vertical velocity $w_b$:

In [ ]:
inv_w = 1        # turn basal velocity inversion 'on'
inv_beta = 0     # turn basal drag inversion 'off'

We are going to make some synthetic data for the example inversion.
This is done by prescribing an oscillating Gaussian anomaly of the form 
$$ w_b^\mathrm{true}(x,y,t) = 5\exp\left(-\frac{x^2+y^2 }{2\sigma^2}\right)\sin(2\pi t\,/\,T) $$
where $T=10$ yr is the final time and $\sigma = 20/3$ km determines the width of the anomaly.

For later comparison, we will want this "true" inverse solution defined above, so we obtain that via:

In [ ]:
from synthetic_data import make_fields
sol_true = make_fields(inv_w,inv_beta)

The "true" elevation is computed by application of the forward operator $\mathcal{H}_{w_b}$:
$$h^\mathrm{true} = \mathcal{H}_{w_b}(w_b^\mathrm{true}) $$
and the synthetic data is constructed via
$$h^\mathrm{obs} = h^\mathrm{true} + \text{noise}.$$
The magnitude of the noise is set by the $\texttt{noise}\_\texttt{level}$ parameter, which determines the deviation from
the smooth elevation by the relative "error"
$$\|h^\mathrm{obs}-h^\mathrm{true} \|/\|h^\mathrm{true}\| = \texttt{noise}\_\texttt{level}.$$
Here the norm over space and time is defined via
$$\|f\|^2  = \int_0^T\int_{-\infty}^{+\infty}\int_{-\infty}^{+\infty} |f(x,y,t)|^2\;\mathrm{d}x\,\mathrm{d}y\,\mathrm{d}t,$$
where obviously the infinite spatial domain is replaced by a "large enough" box. 

In [ ]:
from synthetic_data import make_data

noise_level = 0.01                              # noise level (scaled relative to elevation anomaly norm)

data = make_data(inv_w,inv_beta,noise_level)    # make the synthetic data 

The least-squares inverse solution is obtained by solving the normal equation
$$ \mathcal{H}_{w_b}^\dagger(\mathcal{H}_{w_b}(w_b)) + \mathcal{R}'(w_b) = \mathcal{H}_{w_b}^\dagger (h^\mathrm{obs}) $$
with the conjugate gradient method, where $\mathcal{R}'$ is a regularization term. An analogous equation is used for the basal drag coefficient ($\beta$) inversion. In these examples, we choose an $H^1$-type regularization of the form
$$ \mathcal{R}'(w_b) = -\varepsilon\nabla^2 w_b$$
where $\varepsilon$ is the regularization parameter. 

The goal now is to determine the optimal regularization parameter $\varepsilon$ that minimizes the misfit without overfitting the data.

We are not using surface velocity data for these examples, so we set the velocity "locations" all to zero:

In [ ]:
vel_locs = np.zeros(np.shape(data[0]),dtype=int)

To find the optimal regularization parameter ($\varepsilon$), we will test a range of values, then
pick the one that minimizes the misfit without overfitting the data:

In [ ]:
eps_w = np.array([1e-2,1e-1,1e0,1e1,1e2])     # array of regularization parameters
mis_w = np.zeros(np.shape(eps_w))             # array of misfits

The $\texttt{main}$ function returns the inverse solution $\texttt{sol}$ ($w_b$ in this case), as well as the associated forward solution $\texttt{fwd}$ ($h$ in this case), and the relative misfit $\texttt{mis}=\|h^\mathrm{obs}-h \|/\|h^\mathrm{obs}\|$. 

Convergence information is printed during the conjugate gradient iterations. 

In [ ]:
from main import main

In [ ]:
for i in range(np.size(eps_w)):
    print('------------- testing eps =  '+str(eps_w[i])+' -------------')
    sol,fwd,mis_w[i] = main(data,vel_locs,inv_w,inv_beta,eps_w=eps_w[i],eps_beta=0);
    print('||h-h_obs||/||h_obs|| = '+str(mis_w[i])+' (target = '+str(noise_level)+') \n')

We now determine the optimal paramter via interpolation and root finding:

In [ ]:
from scipy.interpolate import interp1d
mis_w_int = interp1d(eps_w,mis_w,kind='linear')

In [ ]:
from scipy.optimize import root_scalar

eps_w_opt = root_scalar(lambda x: mis_w_int(x)-noise_level,x0=eps_w[0],x1=eps_w[-1]).root

We will plot the "L-curve" later, but first let's see what the optimal inverse solution looks like:

In [ ]:
sol,fwd,mis = main(data,vel_locs,inv_w,inv_beta,eps_w=eps_w_opt,eps_beta=0);
from plotting import snapshots,plot_movie
snapshots(data,fwd,sol,sol_true,inv_w,inv_beta)
#plot_movie(data,fwd,sol,sol_true,inv_w,inv_beta)    # uncomment to plot a png at every time step 

Next, we will repeat the same example for the basal drag coefficient ($\beta$) inversion. Here, he assume that a slippery spot emerges and disappeares over the observation time. The "true" field is given by
$$ \beta^\mathrm{true}(x,y,t) = -8\times 10^{-2}\exp\left(-\frac{x^2+y^2 }{2\sigma^2}\right)B(t) $$
where $B$ is a continuous box-type function that controls the emergence and disappearance of the anomaly (see synthetic_data.py).

Omitting the same level of detail as above, we repeat the test for this input below:

In [ ]:
inv_w = 0                                    # turn basal velocity inversion 'off'
inv_beta = 1                                 # turn basal drag inversion 'on'
sol_true = make_fields(inv_w,inv_beta)       # get the "true" inverse solution
data = make_data(inv_w,inv_beta,noise_level) # create the data
eps_b = np.array([1e2,1e3,1e4,1e5,1e6])      # array of regularization parameters
mis_b = np.zeros(np.shape(eps_b))            # array of misfits

for i in range(np.size(eps_b)):
    print('------------- testing eps =  '+str(eps_b[i])+' -------------')
    sol,fwd,mis_b[i] = main(data,vel_locs,inv_w,inv_beta,eps_beta=eps_b[i],eps_w=0);
    print('||h-h_obs||/||h_obs|| = '+str(mis_b[i])+' (target = '+str(noise_level)+') \n')

mis_b_int = interp1d(eps_b,mis_b,kind='linear')    # interpolate misfits and find the optimal reg. parameter
eps_b_opt = root_scalar(lambda x: mis_b_int(x)-noise_level,x0=eps_b[0],x1=eps_b[-1]).root  
print('--------------------------------------------------------------------')
print('Getting inverse solution at optimal regularization parameter value\n')
sol,fwd,mis = main(data,vel_locs,inv_w,inv_beta,eps_beta=eps_b_opt,eps_w=0);
snapshots(data,fwd,sol,sol_true,inv_w,inv_beta)
#plot_movie(data,fwd,sol,sol_true,inv_w,inv_beta)    # uncomment to plot a png at every time step 

Clearly the reconstructed basal drag field has a smaller amplitude than the "true" solution. In the next notebooks, we show how incorporation of velocity data can remedy this issue.

Finaly, we can plot the "L-curve" for both inversion examples:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.axhline(y=noise_level,color='k',linestyle='--',linewidth=2)
plt.plot(eps_w,mis_w,'o-',color='C3',linewidth=2,markersize=8,mec='k',label=r'$w_b$')
plt.plot([eps_w_opt],[mis_w_int(eps_w_opt)],'*',color='C3',markersize=20,mec='k')
plt.plot(eps_b,mis_b,'^-',color='C0',linewidth=2,markersize=8,mec='k',label=r'$\beta$')
plt.plot([eps_b_opt],[mis_b_int(eps_b_opt)],'*',color='C0',markersize=20,mec='k')
plt.annotate(r'noise level',xy=(3e-1,1.1e-2),fontsize=18,color='k')
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().invert_xaxis()
plt.xlabel(r'$\varepsilon$',fontsize=20)
plt.ylabel(r'$\Vert h^\mathrm{obs}-h^\varepsilon \Vert\,/\,\Vert h^\mathrm{obs}\Vert$',fontsize=20)
plt.legend(fontsize=18,loc='upper right')
plt.tight_layout()
plt.savefig('fig3',bbox_inches='tight')
plt.show()
plt.close()